In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import BertTokenizer

In [7]:
# 1. Load dataset
dataset = load_dataset("imdb")

Using the latest cached version of the dataset since imdb couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at C:\Users\Garry\.cache\huggingface\datasets\imdb\plain_text\0.0.0\e6281661ce1c48d982bc483cf8a173c1bbeb5d31 (last modified on Sat Aug 16 18:34:05 2025).


In [8]:
# 2. Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 3. Encode texts with tokenizer
def encode_with_bert(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding='max_length',
        max_length=128,
    )
    return {
        "input_ids": tokens["input_ids"],
        "attention_mask": tokens["attention_mask"],
        "label": example["label"]
    }

encoded_dataset = dataset.map(encode_with_bert, batched=True)
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


In [9]:
# 4. Prepare DataLoader
train_loader = DataLoader(encoded_dataset["train"], batch_size=32, shuffle=True)
test_loader = DataLoader(encoded_dataset["test"], batch_size=32)

# 5. Define RNN-based model (without LoRA)
class StandardRNNCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, h):
        return torch.tanh(self.i2h(x) + self.h2h(h))

class TextRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn_cell = StandardRNNCell(embed_dim, hidden_dim)
        self.hidden_dim = hidden_dim
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        batch_size, seq_len, _ = embedded.size()
        h_t = torch.zeros(batch_size, self.hidden_dim, device=embedded.device)

        for t in range(seq_len):
            h_t = self.rnn_cell(embedded[:, t, :], h_t)

        return self.fc(h_t)


In [10]:
# 6. Initialize model
model = TextRNN(
    vocab_size=tokenizer.vocab_size,
    embed_dim=128,
    hidden_dim=128,
    num_classes=2
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 7. Define optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()


In [12]:
# 8. Train model
for epoch in range(20):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")
torch.save(model.state_dict(), "rnn_imdb.pth")

Epoch 1, Loss: 0.6751
Epoch 2, Loss: 0.6641
Epoch 3, Loss: 0.6380
Epoch 4, Loss: 0.6196
Epoch 5, Loss: 0.6740
Epoch 6, Loss: 0.6470
Epoch 7, Loss: 0.6302
Epoch 8, Loss: 0.6292
Epoch 9, Loss: 0.6123
Epoch 10, Loss: 0.5936
Epoch 11, Loss: 0.5736
Epoch 12, Loss: 0.5631
Epoch 13, Loss: 0.5382
Epoch 14, Loss: 0.5270
Epoch 15, Loss: 0.5094
Epoch 16, Loss: 0.5194
Epoch 17, Loss: 0.5102
Epoch 18, Loss: 0.5075
Epoch 19, Loss: 0.4697
Epoch 20, Loss: 0.4461


In [13]:
# 9. Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids)
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {correct / total:.2%}")



Test Accuracy: 54.25%
